In [2]:
import numpy as np
import pecos as pc

Definitions:
* **Fault**: Pauli-error on position (t,qb) in circuit
* **Error**: Resulting Pauli-error on the output state
* **Failure**: Change of logical output state

**Task**: Verification that faults can propagate to become errors

In [5]:
# 1. CNOT: verification of fault propagation

n_qbs = 2

# Ideal circuit (without faults)
circ = pc.circuits.QuantumCircuit()
circ.append('CNOT', {(0,1)})

for op in ['X','Y','Z']:
    for qb in range(n_qbs):
        
        # create single qb error (fault)
        err_circ = pc.error_gens.ErrorCircuits()
        qc = pc.circuits.QuantumCircuit()
        qc.append(op, {qb}) # single qb error
        err_circ.add_circuits(time=0, before_faults=qc)
        
        # simulate propagation of this particular fault through circuit
        state = pc.simulators.PauliFaultProp(n_qbs) # track changes of Pauli ops, init state=II
        circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
        circ_runner.run(state, circ, error_circuits=err_circ) # apply fault with 100% at t=0
        
        # readout faults from state
        faults = state.faults
        print("%s on qb %d caused error %s" %(op, qb, faults))

### We can see clearly the 'forward propagation' of X faults and
### the 'backwards propagation' of Z faults (first and last line)
### These are examples of weight-1 faults causing weight-2 errors

X on qb 0 caused error {'X': {0, 1}, 'Y': set(), 'Z': set()}
X on qb 1 caused error {'X': {1}, 'Y': set(), 'Z': set()}
Y on qb 0 caused error {'X': {1}, 'Y': {0}, 'Z': set()}
Y on qb 1 caused error {'X': set(), 'Y': {1}, 'Z': {0}}
Z on qb 0 caused error {'X': set(), 'Y': set(), 'Z': {0}}
Z on qb 1 caused error {'X': set(), 'Y': set(), 'Z': {0, 1}}


Mølmer–Sørensen n-qubit gate: <br>
$MS_{\phi}(\theta)=exp(-i \frac{\phi}{4} S_{\phi}^2)$ with $S_{\phi} = \sum_{i=1}^{n}cos(\phi)X_i + sin(\phi)Y_i$ <br>
* We use n=2 qubits, $\phi=0$ and $\theta=\pi$: $MS_0(\phi)=exp(-i\frac{\pi}{4}X_1X_2)$

In [6]:
# 2. MS: verification of fault propagation

n_qbs = 2

# Ideal circuit (without faults)
circ = pc.circuits.QuantumCircuit()
circ.append('MS', {(0,1)})

for op in ['X','Y','Z']:
    for qb in range(n_qbs):
        
        # create single qb error (fault)
        err_circ = pc.error_gens.ErrorCircuits()
        qc = pc.circuits.QuantumCircuit()
        qc.append(op, {qb}) # single qb error
        err_circ.add_circuits(time=0, before_faults=qc)
        
        # simulate propagation of this particular fault through circuit
        state = pc.simulators.PauliFaultProp(n_qbs) # track changes of Pauli ops, init state=II
        circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
        circ_runner.run(state, circ, error_circuits=err_circ) # apply fault with 100% at t=0
        
        # readout faults from state
        faults = state.faults
        print("%s on qb %d caused error %s" %(op, qb, faults))

### Also here we see weight-2 errors caused by either Y or Z errors on any qubit.

X on qb 0 caused error {'X': {0}, 'Y': set(), 'Z': set()}
X on qb 1 caused error {'X': {1}, 'Y': set(), 'Z': set()}
Y on qb 0 caused error {'X': {1}, 'Y': set(), 'Z': {0}}
Y on qb 1 caused error {'X': {0}, 'Y': set(), 'Z': {1}}
Z on qb 0 caused error {'X': {1}, 'Y': {0}, 'Z': set()}
Z on qb 1 caused error {'X': {0}, 'Y': {1}, 'Z': set()}
